## Instituto Federal da Paraíba
## Disciplina: Mineração de Dados
## Professor: Paulo Ribeiro
## Alunos: Filipe Salomão, Hércules de Sousa e Matheus Alves
##
## Projeto: Classificação de Gêneros Musicais

Aqui importamos a biblioteca Librosa. Essa biblioteca é uma ferramenta de processamento de áudio em Python. Ela fornece uma série de funções e utilitários para análise de áudio e extração de características de arquivos de áudio. É uma biblioteca amplamente utilizada em tarefas de processamento de áudio, como análise de música, reconhecimento de fala e aplicações de aprendizado de máquina relacionadas ao áudio.

In [1]:
import itertools
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.stats import kurtosis
from scipy.stats import skew

Este código define uma função chamada _get_features_ que recebe um sinal de áudio _y_, uma taxa de amostragem _sr_ e alguns parâmetros opcionais relacionados à análise de áudio, como o tamanho da janela de transformação de Fourier (_n_fft_) e o tamanho do salto entre as janelas (_hop_length_).

A função tem como objetivo extrair várias características do sinal de áudio usando a biblioteca Librosa e retornar essas características como um dicionário.

Várias funções do Librosa são chamadas para extrair características específicas do sinal de áudio, como centro espectral, roll-off espectral, taxa de cruzamento por zero, RMS, força de início, contraste espectral, largura de banda espectral e planicidade espectral. O resultado de cada função é uma matriz multidimensional que é convertida em um vetor unidimensional usando o método _.ravel_.

Também as características MFCC (Mel-frequency Cepstral Coefficients) são calculadas usando a função _librosa.feature.mfcc_.

Uma função interna chamada _get_moments_ é definida. Essa função calcula várias estatísticas descritivas para cada descritor, como valor máximo, valor mínimo, média, desvio padrão, curtose e assimetria.

In [2]:
def get_features(y, sr, n_fft=1024, hop_length=512):
    features = {}

    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_rolloff'] = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zero_crossing_rate'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rms'] = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['onset_strength'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['spectral_contrast'] = librosa.feature.spectral_contrast(y=y, sr=sr).ravel()
    features['spectral_bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_flatness'] = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length).ravel()

    mfcc = librosa.feature.mfcc(y=y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)

    for index, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(index)] = v_mfcc.ravel()

    def get_moments(descriptors):
        result = {}

        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)

        return result

    additional_features = get_moments(features)
    additional_features['tempo'] = librosa.feature.tempo(y=y, sr=sr)[0]

    return additional_features

Esta função recebe dois argumentos: _genres_directory_, que é o diretório principal contendo subdiretórios para diferentes gêneros musicais, e _genres_, que é uma lista dos gêneros a serem processados.

A função tem como objetivo percorrer os arquivos de áudio em cada subdiretório de gênero e extrair características desses arquivos usando a função _get_features_ definida acima.

In [3]:
def process_genres_directory_files(genres_directory, genres):
    all_features = []

    for genre in genres:
        genre_directory = f'{genres_directory}/{genre}/wav_files'

        for file in os.listdir(genre_directory):
            file_path = f"{genre_directory}/{file}"

            y, sr = librosa.load(file_path)

            features = get_features(y, sr)

            features['label'] = genre

            all_features.append(features)

    return all_features

Diretório de previews:

In [4]:
genres_directory = './previews'

Lista de gêneros contidos no diretório:

In [5]:
genres = os.listdir(genres_directory)

In [6]:
genres

['forró', 'funk', 'jazz', 'rap', 'rock', 'samba', 'sertanejo']

Processando todos os arquivos de áudio contidos nos subdiretórios:

In [7]:
%%time

all_features = process_genres_directory_files(genres_directory, genres)

CPU times: total: 10min 49s
Wall time: 13min 38s


Transformando as características em um DataFrame:

In [8]:
df_features = pd.DataFrame(all_features)

In [9]:
df_features.shape

(840, 128)

In [10]:
df_features.head()

,spectral_centroid_max,spectral_centroid_min,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_kurtosis,spectral_centroid_skew,spectral_rolloff_max,spectral_rolloff_min,spectral_rolloff_mean,spectral_rolloff_std,...,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo,label
0,4737.508153,635.664676,2098.726133,556.496663,2.887029,1.283600,9130.078125,925.927734,4514.561770,1372.578815,...,-0.086715,-0.088010,32.658615,-31.948511,-5.576648,9.920987,-0.200280,0.234563,143.554688,forró
1,5339.076724,1019.369702,2294.895607,604.767987,1.803070,0.943630,9130.078125,1722.656250,4914.999411,1341.457851,...,0.120776,-0.207388,21.607838,-36.618702,-4.257775,9.034795,-0.080838,0.118976,143.554688,forró
2,6613.348219,1526.374864,2857.957595,685.797071,3.499504,1.245568,9474.609375,2648.583984,6027.380220,1304.601244,...,-0.253195,0.186958,32.782486,-35.293335,-7.233448,11.249509,-0.106279,0.236294,151.999081,forró
3,6935.364306,945.116845,2439.278264,730.595169,4.671441,1.223784,9840.673828,1184.326172,5551.766429,1564.960933,...,0.052664,-0.347261,28.608442,-36.924839,-3.531663,9.956684,0.135261,-0.020543,107.666016,forró
4,7086.838756,928.772862,2541.888664,744.284935,2.716824,1.082830,9969.873047,1162.792969,6118.729668,1422.390345,...,-0.254407,-0.014688,29.134048,-33.681553,-3.894992,9.132825,0.035545,0.014757,135.999178,forró


## Sobre as características do sinal de áudio

### Centroide espectral

A centroide espectral é uma medida que descreve onde está localizada a "média" das frequências de um sinal de áudio. Ela pode ser usada para entender a distribuição das frequências e ajudar a identificar o estilo ou gênero musical de uma música.

Ao calcular a centroide espectral de diferentes músicas de um mesmo gênero musical, você pode observar padrões consistentes. Por exemplo, em músicas de jazz, a centroide espectral pode tender a ser mais baixa, pois há mais ênfase em instrumentos de baixa frequência, como contrabaixo e piano. Em músicas de rock, a centroide espectral pode ser mais alta, pois há mais presença de guitarras elétricas e vocais com frequências mais altas.

### Rolloff

O rolloff espectral é definido como a frequência abaixo da qual uma certa porcentagem (geralmente 85%) da energia total do sinal de áudio está contida nas componentes de frequência mais baixas. Essa medida é útil na análise de áudio, pois pode ser usada para inferir informações sobre a "brilho" ou "timbre" do som.

Por exemplo, em gêneros musicais com graves pesados, como o hip-hop, o rolloff espectral pode ser mais baixo, pois há mais energia espectral em frequências mais baixas. Em contraste, em gêneros musicais como a música clássica, que geralmente não tem muitos graves, o rolloff espectral pode ser mais alto, pois há menos energia espectral em frequências mais baixas.

### Taxa de cruzamento por zero

A taxa de cruzamento por zero é uma medida que indica a quantidade de vezes que o sinal atravessa o eixo horizontal (valor zero) em relação ao tempo. Ela pode ser muito útil ao indicar informações sobre "aspereza" ou "textura".

Música clássica e jazz tendem a ter uma taxa de cruzamento por zero mais baixa, pois geralmente possuem notas sustentadas e menos mudanças rápidas de polaridade.

Música eletrônica e rock podem ter uma taxa de cruzamento por zero mais alta, devido a batidas mais intensas e rápidas, levando a mais mudanças de polaridade em um curto espaço de tempo.

Música acústica ou folk pode ter uma taxa de cruzamento por zero intermediária, pois geralmente possui instrumentos acústicos que produzem um equilíbrio entre mudanças de polaridade rápidas e sustentações de notas.

### RMS

O RMS (Root Mean Square) é uma medida que calcula a média quadrática das amplitudes de um sinal de áudio. Basicamente, o RMS nos dá uma ideia de quão alto ou suave um sinal de áudio é em termos de intensidade.

Música clássica e música ambiente geralmente têm um valor de RMS mais baixo, pois tendem a ser mais suaves e tranquilas em termos de intensidade sonora.

Gêneros musicais como rock, metal ou música eletrônica tendem a ter um valor de RMS mais alto, pois são mais intensos e apresentam uma ampla faixa dinâmica.

Músicas de dança ou hip-hop, com suas batidas pulsantes e fortes, também podem ter um valor de RMS mais alto devido à sua natureza energética e impactante.

### Força do ataque (onset strength)

A força do ataque é uma medida que indica a intensidade ou saliência de mudanças de atividade rítmica no sinal de áudio.

Gêneros musicais como música clássica e jazz costumam ter ataques mais suaves e gradualmente crescentes. A força do ataque nesses gêneros tende a ser menor e menos pronunciada em comparação com outros estilos musicais.

Em gêneros como rock, música eletrônica ou hip-hop, que são conhecidos por suas batidas enérgicas e fortes, a força do ataque é geralmente mais alta. Isso ocorre porque esses estilos musicais enfatizam os momentos de início das notas e têm uma presença percussiva mais marcante.

Gêneros musicais como música clássica contemporânea ou experimental podem apresentar uma variação maior na força do ataque, com eventos sonoros inesperados e mudanças dramáticas na intensidade ao longo da música.

### Contraste espectral de um sinal

O contraste espectral é uma medida que indica a relação entre a energia espectral em bandas de frequência adjacentes.

No forró, o contraste espectral pode revelar a presença marcante de instrumentos como a sanfona e o triângulo em frequências específicas. Esses instrumentos tendem a criar um contraste espectral mais alto, destacando-se em relação a outros elementos sonoros presentes na música.

No rap, a voz do rapper é um elemento central. O contraste espectral pode revelar a clareza e a proeminência da voz em relação aos demais componentes instrumentais. A voz geralmente ocupa uma faixa de frequência mais destacada, enquanto os instrumentos de fundo podem ter um contraste espectral mais baixo.

### Largura de banda espectral

A largura de banda espectral é uma medida que indica a extensão das frequências presentes no sinal de áudio.

No samba, a largura de banda espectral pode ser mais concentrada em frequências percussivas, como o surdo e o pandeiro, que são instrumentos característicos desse estilo musical. Outros instrumentos, como o cavaquinho e o violão, podem ocupar faixas de frequência mais específicas.

No sertanejo, a largura de banda espectral pode variar dependendo da abordagem musical específica. Em arranjos mais tradicionais, a largura de banda pode ser mais moderada, destacando-se as frequências vocais e dos instrumentos de corda, como o violão. Já em arranjos mais modernos, podem ser adicionados elementos eletrônicos e uma maior variação de frequências.

### Nivelamento espectral

O nivelamento espectral, também conhecido como equalização, é um processo que ajusta a intensidade de diferentes frequências em um sinal de áudio. Ele é usado para equilibrar o volume ou a ênfase de certas partes do espectro de frequência, tornando o som mais balanceado e agradável aos ouvidos.

No funk, o nivelamento espectral pode ser usado para realçar os graves e as batidas percussivas, proporcionando um impacto mais pronunciado nas frequências baixas, onde estão concentrados os elementos rítmicos desse gênero musical. Isso ajuda a criar um som mais enérgico e dançante.

No rock, o nivelamento espectral pode ser utilizado para equilibrar os elementos instrumentais, como guitarras, baixos, baterias e vocais. Por exemplo, é possível realçar as frequências das guitarras distorcidas para obter um som mais agressivo, enquanto mantém as frequências vocais claras e bem audíveis. O objetivo é criar uma mistura equilibrada e potente.

Guardando as características em um arquivo CSV:

In [11]:
df_features.to_csv('./all_features.csv', index=False)